In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as path
from matplotlib import colormaps as cm
from matplotlib.colors import LinearSegmentedColormap

## Plotting and contour helpers

In [ ]:
teal = (47.0/255.0, 95.0/255.0, 82.0/255.0)
red = (161.0/255.0, 41.0/255.0, 47.0/255.0)
orange = (224.0/255.0, 105.0/255.0, 63.0/255.0)
blue = (88.0/255.0, 143.0/255.0, 188.0/255.0)
yellow = (230.0/255.0, 215.0/255.0, 20.0/255.0)
purple = (120.0/255.0, 15.0/255.0, 100.0/255.0)

def make_bicolor(cstart, cend, name='bicolor'):
    rgb = [[(0, cstart[i], cstart[i]), (1, cend[i], cend[i])] for i in range(3)]
    return LinearSegmentedColormap(name, segmentdata={'red':rgb[0], 'green':rgb[1], 'blue':rgb[2]}, N=256)

def make_tricolor(cstart, cmid, cend, midpoint=0.5, name='tricolor'):
    rgb = [[(0, cstart[i], cstart[i]), (midpoint, cmid[i], cmid[i]), (1, cend[i], cend[i])] for i in range(3)]
    return LinearSegmentedColormap(name, segmentdata={'red':rgb[0], 'green':rgb[1], 'blue':rgb[2]}, N=256)

bicolor = make_bicolor(red, yellow)
tricolor = make_tricolor(purple, orange, yellow, midpoint=0.7)
viridis = cm.get_cmap('viridis')

In [ ]:
def linf_circle(x_c=0., y_c=0., r=1.):
    '''
    Parametric x, y points to represent a 'circle' in L^\infty
    '''
    x = [x_c-r, x_c-r, x_c+r, x_c+r, x_c-r]
    y = [y_c-r, y_c+r, y_c+r, y_c-r, y_c-r]
    return x, y

def linf_path(start, end):
    '''
    Parametric x, y points to represent a 'distance' path in L^\infty.

    Represented as a broken line connected to both points
    - parallel to the axis of greatest difference
    - line length represents distance
    '''
    startx, starty = start
    endx, endy = end
    if np.abs(startx - endx) > np.abs(starty - endy):
        midx = startx + (endx - startx)/2
        dy = (endy - starty)/20
        return [startx, midx, midx] + [None] + [midx, midx, endx], [starty, starty, starty+dy] + [None] + [endy-dy, endy, endy]
    else:
        midy = starty + (endy - starty)/2
        dx = (endx - startx)/20
        return [startx, startx, startx+dx] + [None] + [endx-dx, endx, endx], [starty, midy, midy] + [None] + [midy, midy, endy]
    
def l2_circle(x_c=0., y_c=0., r=1.):
    '''
    Parametric x, y points to represent a circle in L^2
    '''
    t = np.linspace(-np.pi, np.pi, 100)
    x = x_c + r * np.cos(t)
    y = y_c + r * np.sin(t)
    return x, y

def l2_path(start, end):
    '''
    Parametric x, y points to represent a distance path in L^2
    '''
    return list(zip(start, end))

def l1_circle(x_c=0., y_c=0., r=1.):
    '''
    Parametric x, y points to represent a 'circle' in L^1
    '''
    x = [x_c-r, x_c, x_c+r, x_c, x_c-r]
    y = [y_c, y_c+r, y_c, y_c-r, y_c]
    return x, y

def l1_path(start, end):
    '''
    Parametric x, y points to represent a distance path in L^2.
    
    Represented as a bent line taking each axis in turn
    '''
    return [start[0], start[0], end[0]], [start[1], end[1], end[1]]

In [ ]:
def distance_map(norm_split, point, minx=-1, maxx=1, miny=-1, maxy=1):
    '''
    Produce a grid map of norm distance from a given point
    :param norm_split: 'split' norm function taking xdiff, ydiff as parameters
    :param point: xy position of point to measure from
    '''
    xs = np.linspace(minx, maxx, 100)
    ys = np.linspace(miny, maxy, 100)
    px, py = point
    xx, yy = np.meshgrid(xs-px, ys-py, sparse=True)
    return norm_split(xx, yy)

def linf_split(xdists, ydists):
    return np.maximum(np.abs(xdists), np.abs(ydists))

def l2_split(xdists, ydists):
    return np.sqrt(xdists**2 + ydists**2)

def l1_split(xdists, ydists):
    return np.abs(xdists) + np.abs(ydists)

## Producing graphics for alignment metrics

### Mixing different distance/normalisation

In [ ]:
def plot_mixed_norms_opposites(start, other, circle_normalisation, path_distance, norm_distance, norm=None):
    '''
    A pretty illustration of using different norm for distance and normalisation
    :param start: identified point to measure distance from
    :param other: another point, not opposite, to illustrate distance
    :param circle_normalisation: function to produce circle representation for the normalisation norm
    :param path_distance: function to produce distance path representation for the distance norm
    :param norm_distance: function to measure the distance norm given diffx and diffy
    :param norm: function to measure the normalisation
    '''
    if norm is not None:
        start = start / norm(*start)
        other = other / norm(*other)

    unit_circle = circle_normalisation()
    ds = distance_map(norm_distance, start)

    opp_path = path_distance(start, -start)
    other_path = path_distance(start, other)

    fig, ax = plt.subplots()
    ax.plot(*unit_circle, '--', c='black', label='unit circle (normalisation)')
    im = ax.imshow(ds, extent=[-1, 1, -1, 1], origin='lower', cmap=tricolor)
    clip = path.Path(np.stack(unit_circle, -1))
    im.set_clip_path(clip, transform=ax.transData)
    fig.colorbar(im, label='distance')

    ax.scatter(*start, c='black', label='true opposites')
    ax.scatter(*-start, c='black')
    ax.scatter(*other, c=blue, label='further (distance)')

    ax.plot(*opp_path, ':', c='black')
    ax.plot(*other_path, ':', c=blue)

    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    return fig, ax

A plot of the 2d $L^\infty$ standard surface (a square), with $L^2$ distances, showing

- a pair of opposite points having one distance
- a third, non-opposite point having _greater_ ($L^2$) distance

In [ ]:
_, ax = plot_mixed_norms_opposites(
    start=np.array([-1, 0]),
    other=np.array([1, -0.5]),
    circle_normalisation=linf_circle,
    path_distance=l2_path,
    norm_distance=l2_split)
ax.set_title('$L^2$ distance with $L^\infty$ normalisation')
ax.legend(loc='upper left');

A plot of the 2d $L^2$ standard surface (a circle), with $L^\infty$ distances, showing

- a pair of opposite points having one distance
- a third, non-opposite point having _greater_ ($L^\infty$) distance

In [ ]:
_, ax = plot_mixed_norms_opposites(
    start=np.array([-1, -1]),
    other=np.array([1, -0.05]),
    circle_normalisation=l2_circle,
    path_distance=linf_path,
    norm_distance=linf_split,
    norm=l2_split)
ax.set_title('$L^\infty$ distance with $L^2$ normalisation')
ax.legend(loc='upper left');

### Strict convexity and opposites

In [ ]:
def plot_maximal_and_opposites(start, other, circle_norm, path_norm, norm=None, reverse_other_path=False, show_other_projected=False):
    '''
    A pretty illustration of maximal distances and opposite points.
    
    Expects same norm to be used for distance and normalisation.

    :param start: identified point to take distances from
    :param other: another, non-opposite point, to illustrate distance
    :param circle_norm: function to produce circle representation
    :param path_norm: function to produce distance path representation
    :param norm: function to measure the norm
    :param reverse_other_path: draw the path from start to other in the reverse direction
    :param show_other_projected: project the other point away from start, to maximal distance, and show additionally
    '''
    extent = [-1.3, 1.3, -1.3, 1.3]
    
    unit_circle = circle_norm()
    if norm is not None:
        start = start / norm(*start)
    opp = -start
    if show_other_projected:
        other_projected = start + 2*(other - start)/norm(*(other - start))

    ds = distance_map(norm, start, *extent)
    maximal_circle = circle_norm(*start, 2)

    opp_path = path_norm(start, opp)
    other_path = path_norm(start, other) if not reverse_other_path else path_norm(other, start)

    fig, ax = plt.subplots(1, 1, figsize=(5, 5))

    im = ax.imshow(ds, extent=extent, origin='lower', cmap=tricolor, vmax=2)
    clip = path.Path(np.stack(maximal_circle, -1))
    im.set_clip_path(clip, transform=ax.transData)
    fig.colorbar(im, label='distance')

    ax.plot(*unit_circle, '--', c='black', label='unit circle')
    ax.plot(*maximal_circle, c='black', label='maximal-distance circle')

    ax.scatter(*start, c='black', label='true opposites')
    ax.scatter(*opp, c='black')
    ax.scatter(*other, c=blue, label='non-opposite')
    if show_other_projected:
        ax.scatter(*other_projected, marker='^', c=blue, label='projected')

    ax.plot(*opp_path, ':', c='black')
    ax.plot(*other_path, ':', c=blue)

    ax.set_xlim(*extent[:2])
    ax.set_ylim(*extent[2:])
    return ax

A plot of the $L^2$ unit circle, showing

- an identified point with a 'maximal distance' circle around it
- an opposite point, maximally distant
- a third point, not maximally distant

In [ ]:
ax = plot_maximal_and_opposites(
    start=np.array([-1, -1]),
    other=np.array([1, 0]),
    circle_norm=l2_circle,
    path_norm=l2_path,
    norm=l2_split,
    show_other_projected=True)
ax.set_title('$L^2$ (strictly convex)')
ax.legend(loc='upper left');

A plot of the $L^\infty$ unit circle, showing

- an identified point with a 'maximal distance' circle around it
- an opposite point, maximally distant
- a third point, also maximally distant

In [ ]:
ax = plot_maximal_and_opposites(
    start=np.array([-1, 0]),
    other=np.array([1, -0.5]),
    circle_norm=linf_circle,
    path_norm=linf_path,
    norm=linf_split,
    reverse_other_path=True) # draw the path starting from the end, for better visual distinction
ax.set_title('$L^\infty$ (not strictly convex)')
ax.legend(loc='upper left');

A plot of the $L^1$ unit circle, showing

- an identified point with a 'maximal distance' circle around it
- an opposite point, maximally distant
- a third point, also maximally distant

In [ ]:
ax = plot_maximal_and_opposites(
    start=np.array([-1, 0]),
    other=np.array([0.5, -0.5]),
    circle_norm=l1_circle,
    path_norm=l1_path,
    norm=l1_split)
ax.set_title('$L^1$ (not strictly convex)')
ax.legend(loc='upper left');